In [83]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms, utils
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import cv2
import pandas as pd
import pydicom as dicom
from matplotlib.pyplot import imshow
import torch.optim as optim
import numpy as np
from PIL import Image


from glob import glob
import os

In [2]:
if torch.cuda.is_available():
    # you can continue going on here, like cuda:1 cuda:2....etc.
    device = torch.device("cuda:0")
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [3]:
DATA_DIR = os.path.join(".", "rsa-pneumonia-data")

TRAIN_IMAGES = os.path.join(DATA_DIR, "stage_2_train_images")
TEST_IMAGES = os.path.join(DATA_DIR, "stage_2_test_images")

TRAIN_ANNOTATIONS_FILE = "stage_2_train_labels.csv"
TRAIN_CLASS_FILE = "stage_2_detailed_class_info.csv"

In [70]:
class OpacityDataset(Dataset):

    def __init__(self, csv_file, root_dir, image_dir, transform=None):
        self.classes_df = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.image_dir = image_dir
        self.transform = transform

        # Convert our classes to integers!
        self.class_dict = {
            "Normal": 0,
            "No Lung Opacity / Not Normal": 1,
            "Lung Opacity": 1
        }

    def __len__(self):
        return len(self.classes_df)

    def __getitem__(self, index):
        if torch.is_tensor(index):
            index = index.tolist()

        # Get the image, preprocess it for our model
        image_path = os.path.join(
            self.image_dir, self.classes_df.iloc[index, 0])
        image = dicom.read_file(image_path + ".dcm").pixel_array
        image = image[::4, ::4]
        image = image/image.max()
        image = (255*image).clip(0, 255).astype(np.uint8)
#         image /= 255.0
#         image = image/image.max()
        image = Image.fromarray(image).convert("RGB")
        opacity_class = self.classes_df.at[index, 'class']

        if self.transform:
            image = self.transform(image)
        
        sample = {"image": image,
                  "opacity_class": self.class_dict[opacity_class]}

        return sample

In [77]:
opacity_dataset = OpacityDataset(
    csv_file=os.path.join(DATA_DIR, TRAIN_CLASS_FILE),
    root_dir=DATA_DIR,
    image_dir=TRAIN_IMAGES,
    transform=transforms.ToTensor()
)

# for i in range(len(opacity_dataset)):
#     sample = opacity_dataset[i]
#     imshow(sample["image"][0])
#     print(sample["opacity_class"])
#     break

In [60]:
# Define our NN
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, (3, 3))
        self.pool = nn.MaxPool2d((2, 2), 2)
        self.conv2 = nn.Conv2d(16, 32, (3, 3))
        self.conv3 = nn.Conv2d(32, 64, (3, 3))
        self.conv4 = nn.Conv2d(64, 128, (3, 3))
        self.linear1 = nn.Linear(25088, 4086)
        self.linear3 = nn.Linear(4086, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = x.view(x.size()[0], -1)
        x = F.relu(self.linear1(x))
        x = self.linear3(x)
        return x
    
class AlexNet(nn.Module):
    def __init__(self, num_classes=2):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 7 * 7)
        x = self.classifier(x)
        return x

model = AlexNet()
model.to(device)

AlexNet(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=12544, out_features=4096, bias=True)
    (2): ReLU(inplace=True)
    (3): Dropout(p=0.5,

In [7]:
del model
torch.cuda.ipc_collect()
torch.cuda.empty_cache()

In [78]:
# Hyperparameters
epochs = 5
num_classes = 2
batch_size = 32
learning_rate = .001

In [88]:
model = torchvision.models.resnet18(pretrained=True)
feature_num = model.fc.in_features
model.fc = nn.Linear(feature_num, 2)
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [89]:
train_loader = torch.utils.data.DataLoader(
    opacity_dataset, batch_size=batch_size, shuffle=True, num_workers=0)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)

In [90]:
def multi_acc(y_pred, y_test):
    y_pred_softmax = torch.log_softmax(y_pred, dim=1)
    _, y_pred_tags = torch.max(y_pred_softmax, dim=1)

    correct_pred = (y_pred_tags == y_test).float()
    acc = correct_pred.sum() / len(correct_pred)
    acc = acc * 100

    return acc

In [ ]:
for epoch in range(epochs):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data["image"], data["opacity_class"]
        inputs, labels = Variable(inputs.cuda(),requires_grad=True), Variable(labels.cuda())

        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        prob = F.softmax(outputs, dim=-1)

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000), " accuracy:", multi_acc(outputs, labels))
            running_loss = 0.0
print('Finished Training')

[1,   100] loss: 0.019  accuracy: tensor(87.5000, device='cuda:0')
[1,   200] loss: 0.017  accuracy: tensor(90.6250, device='cuda:0')
[1,   300] loss: 0.015  accuracy: tensor(90.6250, device='cuda:0')
[1,   400] loss: 0.016  accuracy: tensor(81.2500, device='cuda:0')
